In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.preprocessing import sequence
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, GRU, Embedding
from keras.models import load_model
import time

Using TensorFlow backend.


In [2]:
data1 = pd.read_csv("math_expr.csv",index_col=None)
data = data1.iloc[:50000,].copy()

In [3]:
data.loc[data["Expression Name"]=='distributor_expression','Expression Name']=0
data.loc[data["Expression Name"]=='linear_expression','Expression Name']=1
data.loc[data["Expression Name"]=='linear_equation','Expression Name']=2

In [4]:
start_time = time.time()
a=0
for s in data.iloc[:,0]:
    data.iloc[a,0] = bytearray(s, 'utf-8')
    a+=1
    # print(data.iloc[a,0][0]+"--- {}s seconds ---".format(time.time() - start_time, 2))

In [5]:
X_train,X_test,y_train,y_test = train_test_split(data.iloc[:,0],data.iloc[:,1],test_size=0.2)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(40000,)
(10000,)
(40000,)
(10000,)


In [6]:
X_train_seq = sequence.pad_sequences(X_train, maxlen=100)
X_test_seq = sequence.pad_sequences(X_test, maxlen=100)

In [7]:
Y_train_ctg = np_utils.to_categorical(y_train)
Y_test_ctg = np_utils.to_categorical(y_test)

In [8]:
model = Sequential()
model.add(Embedding(50000, 100))
model.add(GRU(100, activation='tanh'))
model.add(Dense(3, activation='softmax'))

Instructions for updating:
Colocations handled automatically by placer.


In [9]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [10]:
history = model.fit(X_train_seq, Y_train_ctg, batch_size=128, 
                    epochs=10,validation_data=(X_test_seq, Y_test_ctg))

Instructions for updating:
Use tf.cast instead.
Train on 40000 samples, validate on 10000 samples
Epoch 1/10
40000/40000 [==============================] - 45s 1ms/step - loss: 0.2724 - acc: 0.9002 - val_loss: 0.0756 - val_acc: 0.9826
Epoch 2/10
40000/40000 [==============================] - 45s 1ms/step - loss: 0.0640 - acc: 0.9860 - val_loss: 0.0600 - val_acc: 0.9884
Epoch 3/10
40000/40000 [==============================] - 45s 1ms/step - loss: 0.0479 - acc: 0.9898 - val_loss: 0.0413 - val_acc: 0.9884
Epoch 4/10
40000/40000 [==============================] - 45s 1ms/step - loss: 0.0357 - acc: 0.9906 - val_loss: 0.0797 - val_acc: 0.9773
Epoch 5/10
40000/40000 [==============================] - 45s 1ms/step - loss: 0.0346 - acc: 0.9902 - val_loss: 0.0412 - val_acc: 0.9884
Epoch 6/10
40000/40000 [==============================] - 45s 1ms/step - loss: 0.0330 - acc: 0.9908 - val_loss: 0.0401 - val_acc: 0.9884
Epoch 7/10
40000/40000 [==============================] - 45s 1ms/step - loss: 0

In [12]:
print("\n Test Accuracy: %.4f" % (model.evaluate(X_test_seq, Y_test_ctg)[1]))

10000/10000 [==============================] - 4s 366us/step

 Test Accuracy: 0.9884


In [13]:
model.save('MExC_gru_V1.h5')